# 추천 시스템

<아이템간 유사성 파악 방법>
1. 협업 필터링 : 다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성(행동, 관계) 
- 다수 사용자의 판단을 기반으로 정확한 추천을 가능
- 제약사항 : 콜드 스타트(충분한 정보 부족), 계산량이 너무 낳아  추천 효율이 적은 경우, 관심 저조 항목 정보 부족으로 추천에서 배제
2.  콘텐츠 기반 필터링 : 아이템 고유의 정보(속성)를 바탕

<기타>
- 추천 시스템에서는 검증을 위하여 적용하는 분야에 대한 지식이 더 필요함

### 1. 데이터 준비

In [2]:
# 판다스를 활용하여 데이터 확인, 컬럼명 col_names에 지정

import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 컬럼명을 지정 
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [3]:
# 사용할 컬럼 노출
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
# 쉬운 검색을 위해 문자열 소문자로 변경

data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
#1번째 유저의 아티스트 노래 확인

condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


### 2. 데이터 탐색

- 유저수, 아티스트수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

In [6]:
# 유저 수
data['user_id'].nunique()
print ('유저수는 {} 이다.'.format(data['user_id'].nunique()))

유저수는 358868 이다.


In [7]:
# 아티스트 수
data['artist'].nunique()
print ('아티스트 수는 {} 이다.'.format(data['artist'].nunique()))

아티스트 수는 291346 이다.


In [8]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [9]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계

user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [10]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

#### 모델 검증을 위한 사용자 초기 정보 세팅

In [11]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['linkin park' , 'maroon5' ,'jason mraz' ,'coldplay' ,'nirvana']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,linkin park,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,nirvana,30


- my_favorite = ['nirvana' , 'maroon5' ,'jason mraz' ,'coldplay' ,'linkin park']  수정하였지만, LMS 포함 변경이 되지 않음

### 3. 모델에 활용하기 위한 전처리(실습)  
- 데이터에 번호 붙이기 =인덱싱

In [12]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)} #enumerate()  리스트가 있는 경우 순서와 리스트의 값을 전달하는 기능
artist_to_idx = {v:k for k,v in enumerate(artist_unique)} 

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])   

358868
376


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,352,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


### 4. 사용자의 명시적/암묵적 평가
- https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301

In [15]:
# 1회만 play한 데이터의 비율을 보는 코드
# 임의 규칙을 정해본다.

only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


### 5.  Matrix Factorization(MF, 행렬분해  모델) *사용자 특성벡터 *영화 특성 벡터
- https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b

### 6. CSR(Compressed Sparse Row) Matrix
- CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조
- https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix
- https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

In [16]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Row format>

### 7. MF  모델 학습하기
- implicit 패키지 (https://github.com/benfred/implicit) : 암묵적(implicit) dataset을 사용하는 다양한 모델을  빠르게 학습할 수 있는 패키지
- 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용
- Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련 마무리
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [21]:
# Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지 확인
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [22]:
zimin_vector

array([ 0.09890637,  0.322452  ,  0.6561922 , -0.7520876 , -0.08523291,
       -0.99137205,  1.3384081 , -0.09720298, -0.0906347 , -0.18894964,
       -1.050532  ,  0.42489552, -0.03211749,  0.70714265, -0.1636722 ,
        0.44056413, -1.057749  , -1.2097505 , -0.44722813, -0.31291345,
       -1.3533145 ,  0.6813808 ,  0.19580561, -0.02939379, -0.5603959 ,
       -1.1088097 ,  1.1961408 , -1.4043971 ,  0.5714977 , -0.05059278,
       -1.3375242 , -0.00845906, -0.56087476, -1.593319  ,  0.10349883,
        1.016347  ,  0.5133342 ,  0.4755614 ,  0.0097713 ,  0.5791914 ,
        0.37745374,  0.6287552 ,  0.65353394,  0.4262673 ,  0.9386017 ,
        2.295203  , -0.399639  ,  0.2625425 , -0.52240074,  0.2550086 ,
        0.2828115 ,  0.14683472,  0.7196381 ,  0.20912771,  1.1680999 ,
        0.16826573, -0.14484684,  0.20951843,  0.00624947, -0.00874171,
       -0.9780454 , -0.45168447,  1.185709  , -0.26315358, -0.5139304 ,
       -0.11172493, -0.57362056,  0.23723057,  0.10691562,  0.82

In [23]:
black_eyed_peas_vector

array([-7.72154983e-03,  9.10298992e-03,  4.38931817e-03,  6.85951533e-03,
        2.26978911e-03,  1.04837660e-02, -5.27033862e-03,  1.35048078e-02,
        1.91842746e-02, -6.13581622e-03, -7.23400759e-03, -8.83213419e-04,
        2.09888862e-03,  1.02269407e-02,  1.17463002e-03,  1.35168815e-02,
       -2.50878613e-02, -1.88329606e-03,  8.81543476e-03, -1.79506112e-02,
        1.43642658e-02,  8.34117178e-03,  3.34753934e-03, -4.49516717e-03,
       -1.64324399e-02, -8.72083195e-03,  2.92793738e-05, -4.30413801e-03,
        1.52431438e-02, -2.31949100e-03,  1.56408250e-02,  1.11661665e-02,
       -4.33627283e-03,  1.36352435e-03,  7.31073506e-03, -1.41841150e-03,
       -5.65811899e-03,  4.71114134e-03,  2.35593654e-02,  6.89152908e-03,
        1.45840943e-02,  2.44107973e-02,  1.10100619e-02,  1.76912844e-02,
        1.95684880e-02, -4.31729219e-04, -1.06292563e-02,  1.64285824e-02,
        1.27021559e-02,  7.55008217e-03,  1.58787481e-02, -2.16959830e-04,
        1.26480137e-03,  

In [24]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.24121745

In [25]:
# 모델이 나의 queen에 대한 선호도를 어떻게 예측
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.47397727

### 8. 비슷한 아티스트 찾기 + 유저에게 추천하기
- AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾기

In [26]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 1.0),
 (277, 0.9876711),
 (5, 0.9803202),
 (28, 0.9784283),
 (217, 0.97683674),
 (247, 0.97398514),
 (473, 0.972794),
 (490, 0.97274595),
 (418, 0.96682024),
 (268, 0.9610484),
 (910, 0.95756996),
 (1018, 0.9543965),
 (782, 0.9478441),
 (75, 0.94711965),
 (694, 0.9414381)]

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'red hot chili peppers',
 'the killers',
 'radiohead',
 'the beatles',
 'placebo',
 'oasis',
 'u2',
 'pink floyd',
 'nirvana',
 'the smashing pumpkins',
 'the white stripes',
 'queen',
 'foo fighters']

In [28]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [29]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'dr. dre',
 'nas',
 'the game',
 'snoop dogg',
 'ludacris',
 'jay-z',
 'bone thugs-n-harmony',
 '50 cent']

In [30]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'katy perry',
 'rihanna',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'kelly clarkson',
 'justin timberlake',
 'pink']

#### 유저에게 아티스트 추천
- AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받기

In [31]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(5, 0.5383365),
 (277, 0.52491164),
 (247, 0.5105499),
 (28, 0.4932043),
 (217, 0.48839),
 (473, 0.48577428),
 (694, 0.48301542),
 (268, 0.47790965),
 (418, 0.4777553),
 (503, 0.4748699),
 (75, 0.47397727),
 (490, 0.4692092),
 (2113, 0.44631213),
 (24, 0.43529913),
 (193, 0.43172523),
 (773, 0.4296126),
 (531, 0.42785382),
 (283, 0.42572656),
 (55, 0.42367455),
 (1170, 0.42109612)]

In [32]:
[idx_to_artist[i[0]] for i in artist_recommended]

['red hot chili peppers',
 'muse',
 'the beatles',
 'the killers',
 'radiohead',
 'placebo',
 'foo fighters',
 'pink floyd',
 'u2',
 'green day',
 'queen',
 'oasis',
 'incubus',
 'jack johnson',
 'system of a down',
 'nine inch nails',
 'depeche mode',
 'metallica',
 'arctic monkeys',
 'daft punk']

In [33]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [34]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('jason mraz', 0.12082135624556965),
 ('linkin park', 0.06360956407189335),
 ('coldplay', 0.04095143281582832),
 ('nirvana', 0.023794889702380258),
 ('maroon5', -0.00040048443800358246)]

- 객관적인 지표 추천시스템 평가 방법 Mean Average Precision(MAP) 설명 : https://danthetech.netlify.app/DataScience/evaluation-metrics-for-recommendation-system